# 远程手柄遥控 Control DOFBOT by handle
在本例中，我们将使用连接到web浏览器机器的gamepad控制器远程控制机械臂。

In this example, we will use the gamepad controller connected to the web browser machine to remotely control the manipulator.

### 创建手柄控制器 Create handle controller
使用手柄控制机械臂的步骤:
1. 打开[http://html5gamepad.com](http://html5gamepad.com)此网页.  
2. 按下你正在使用的手柄的按键
3. 记住当你按下按键后弹出的相应的索引号

Steps to use the handle to control the mechanical arm:
1. Open[ http://html5gamepad.com ]( http://html5gamepad.com ）This page
2. Press the key of the handle you are using
3. Remember the corresponding index number that pops up when you press the key


In [1]:
import ipywidgets.widgets as widgets
# Replace with the index number of the controller you just tested
# 用你刚测试过正在使用的控制器的索引号替代
controller = widgets.Controller(index=0)  
display(controller)

Controller()

相关模块导入 

Related module import

In [2]:
# Import library path
# 函数库路径导入
import threading
import time
# Thread function operation Library
# 线程功能操作库
import inspect
import ctypes
# Import manipulator objects
# 导入机械臂对象
from Arm_Lib import Arm_Device
Arm = Arm_Device()

创建主动停止进程的方法 

Create a method to actively stop a process

In [3]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# 创建手柄遥杆控制机械臂运动的方法  
## 如果将手柄的模拟模式打开，即红灯亮起时，左边的方向键则不可以用，请使用左右摇杆和LR两边的键来控制机械臂。

## 程序功能: 
1. 左边摇杆和方向键控制一号和二号舵机，其中左右方向控制1号舵机左右运动，上下方向控制2号舵机前后运动。 
2. 右边摇杆和数字键控制五号和六号舵机，其中左右方向控制5号舵机左右翻转，上下方向控制6号舵机夹子夹紧和放松。
3. L1和L2控制3号舵机向前或者向后运行。
4. R1和R2控制4号舵机向前或者向后运行。
5. 按下SELECT键将机械臂所有舵机角度都设置为90度  

# Method of creating handle remote rod to control manipulator movement
## If the simulation mode of the handle is turned on, that is, when the red light is on, the left direction key cannot be used. Please use the left and right rocker and the keys on both sides of LR to control the manipulator.
## Program functions:
1. The left rocker and direction key control No. 1 and No. 2 steering gear, in which the left and right direction controls the left and right movement of No. 1 steering gear, and the up and down direction controls the back and forth movement of No. 2 steering gear.
2. The right rocker and number keys control No. 5 and No. 6 steering gear. The left and right direction controls the left and right turning of No. 5 steering gear, and the up and down direction controls the clamping and loosening of No. 6 steering gear clamp.
3. L1 and L2 control No. 3 steering gear to run forward or backward.
4. R1 and R2 control No. 4 steering gear to run forward or backward.
5. Press the select key to set all steering gear angles of the mechanical arm to 90 degrees


In [4]:
def Arm_Handle():
    s_time = 500
    s_step = 1
    angle_1 = angle_2 = angle_3 = angle_4 =  angle_5 = angle_6 = 90
    while 1:
        #因为摇杆手柄个别差异,所有在遥杆复位值不一定是零，所以需要以0.1作为过滤，避免误操作。
        #Due to the individual differences of the rocker handle, all the reset values of the remote lever are not necessarily zero, so it is necessary to filter with 0.1 to avoid misoperation.
        # 2号舵机，A1上负下正   No. 2 steering gear, A1 upper negative and lower positive
        if controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1:
            time.sleep(.000001)
        else:
            if controller.axes[1].value > 0.1:
                angle_2 += s_step
            else:
                angle_2 -= s_step
            if angle_2 > 180:
                angle_2 = 180
            elif angle_2 < 0:
                angle_2 = 0
            Arm.Arm_serial_servo_write(2, angle_2, s_time)
            time.sleep(0.01)

        # 1号舵机，A0左负有正  No. 1 steering gear, A0 left positive
        if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1):
                time.sleep(.000001)
        else:
            if controller.axes[0].value > 0.1:
                angle_1 -= s_step
            else:
                angle_1 += s_step
                
            if angle_1 > 180:
                angle_1 = 180
            elif angle_1 < 0:
                angle_1 = 0
            Arm.Arm_serial_servo_write(1, angle_1, s_time)
            time.sleep(0.01)

        # 6 steering gear, num1 = B0, Num3 = B2, A2 upper negative and lower positive
        # 6号舵机，NUM1=B0,NUM3=B2, A2上负下正  
        if controller.buttons[0].value == True:
            angle_6 += s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.buttons[2].value == True:
            angle_6 -= s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.axes[2].value > 0.5:
            angle_6 -= s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)
        elif controller.axes[2].value < -0.5:
            angle_6 += s_step
            if angle_6 > 180:
                angle_6 = 180
            elif angle_6 < 0:
                angle_6 = 0
            Arm.Arm_serial_servo_write(6, angle_6, s_time)
            time.sleep(0.01)

        # No. 5 steering gear, num2 = B1, num4 = B3, A5 left positive
        # 5号舵机，NUM2=B1,NUM4=B3, A5左负有正     
        if controller.buttons[1].value == True:
            angle_5 += s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)
            time.sleep(0.01)
        elif controller.buttons[3].value == True:
            angle_5 -= s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)  
            time.sleep(0.01)
        elif controller.axes[5].value > 0.5:
            angle_5 += s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)
            time.sleep(0.01)
        elif controller.axes[5].value < -0.5:
            angle_5 -= s_step
            if angle_5 > 180:
                angle_5 = 180
            elif angle_5 < 0:
                angle_5 = 0
            Arm.Arm_serial_servo_write(5, angle_5, s_time)  
            time.sleep(0.01)


        # 4 steering gear, R1 = B5, R2 = B7
        # 4号舵机，R1=B5,R2=B7   
        if controller.buttons[5].value == True:
            angle_4 -= s_step
            if angle_4 > 180:
                angle_4 = 180
            elif angle_4 < 0:
                angle_4 = 0
            Arm.Arm_serial_servo_write(4, angle_4, s_time)
            time.sleep(0.01)
        elif controller.buttons[7].value == True:
            angle_4 += s_step
            if angle_4 > 180:
                angle_4 = 180
            elif angle_4 < 0:
                angle_4 = 0
            Arm.Arm_serial_servo_write(4, angle_4, s_time)  
            time.sleep(0.01)

        # 3 steering gear, L1 = B4, L2 = B6
        # 3号舵机，L1=B4,L2=B6   
        if controller.buttons[4].value == True:
            angle_3 -= s_step
            if angle_3 > 180:
                angle_3 = 180
            elif angle_3 < 0:
                angle_3 = 0
            Arm.Arm_serial_servo_write(3, angle_3, s_time)
            time.sleep(0.01)
        elif controller.buttons[6].value == True:
            angle_3 += s_step
            if angle_3 > 180:
                angle_3 = 180
            elif angle_3 < 0:
                angle_3 = 0
            Arm.Arm_serial_servo_write(3, angle_3, s_time)  
            time.sleep(0.01)
        
        # Press the selection key B8 to set the steering gear of the manipulator to 90 degrees
        # 按下选择键B8,让机械臂的舵机都设置到90度
        if controller.buttons[8].value == True:
            angle_1 = angle_2 = angle_3 = angle_4 = angle_5 = angle_6 = 90
            Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, 1000)
            time.sleep(1)
            

通过运行下面单元格代码开启手柄实时控制机械臂的进程

Start the handle by running the following cell code to control the process of the manipulator in real time

In [5]:
thread2 = threading.Thread(target=Arm_Handle)
thread2.setDaemon(True)
thread2.start()

通过运行下面单元格代码结束手柄进程，如果出现进程启动或者结束失败的情况，请重新start一下kernel，再重头运行。

End the handle process by running the following cell code. If the process fails to start or end, please restart the kernel and run again.

In [6]:
stop_thread(thread2)